In [2]:
%config Completer.use_jedi = False

%load_ext autoreload
%autoreload 2

In [3]:
SEED = 123
bar_format = '{desc}{percentage:5.1f}%|{bar:10}{r_bar}'

In [4]:
from pprint import pprint
import yaml
from easydict import EasyDict
import pickle

from tqdm import tqdm

from registration_metrics import (
    seed_everything,
    CARLA_dataset,
    prepare_data,
    sampling_pipeline
)

from paths import PROJECT_DIR

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
def run_CARLA_seq5(
    seed, config,
    map_size=10, N_samples=5, cov_scaler=10,
    start_internal_id=0, end_internal_id=2422, step_internal_id=40,
    verbose=False
):
    seed_everything(seed)

    prepare_func = prepare_data

    results = {}
    for internal_id in tqdm(
        range(start_internal_id, end_internal_id, step_internal_id),
        desc="Internal IDs",
        bar_format=bar_format
    ):
        start_id, end_id = internal_id, internal_id + map_size

        data = CARLA_dataset(start_id=start_id, end_id=end_id,
                             dataset_coords_type="world")

        pcs = data.clds
        T_gt = data.poses

        pcs, T_gt, orth_planes, orth_planes_cld_ind, details = prepare_func(
            pcs, T_gt, config)
        details["orth_planes"] = orth_planes
        details["orth_planes_cld_ind"] = orth_planes_cld_ind

        tips = EasyDict(orth_list=orth_planes, config=config)

        save_dict = sampling_pipeline(
            pcs=pcs, T_gt=T_gt, map_tips=tips,
            N_samples=N_samples, cov_scaler=cov_scaler,
            orth_planes_cld_ind=orth_planes_cld_ind,
            verbose=verbose
        )

        results[internal_id, map_size] = {**details, **save_dict}

    down_config = config.orth_planes_extraction.downsample
    voxel_size = down_config.voxel_size
    use_upsampe = down_config.upsample

    filename = f"CARLA-downsampling{{}}_vs{voxel_size}.pkl".format(
        "_upsampling" if use_upsampe else ""
    )
    with open(filename, 'wb') as sfile:
        pickle.dump(results, sfile)
    print(f"Results were saved to \"{filename}\".")

In [6]:
def load_config(filename, verbose=True,
                config_dir=f"{PROJECT_DIR}/configs/mom_pipeline"):
    with open(f'{config_dir}/{filename}.yaml', 'r') as file:
        config = EasyDict(yaml.safe_load(file))

    if verbose:
        pprint(config)

    return config

In [7]:
cfg = load_config("cosine_affinity")
run_CARLA_seq5(SEED, cfg)

{'metrics': {'mean_map_metric': {'min_nn': 6},
             'mom': {'input': 'whole', 'min_nn': 6}},
 'normals_estimation': {'max_nn': 30, 'max_radius': 1.0},
 'orth_planes_extraction': {'affinity': 'cosine',
                            'dist_threshold': 0.1,
                            'downsample': {'upsample': False,
                                           'use': False,
                                           'voxel_size': 0.6},
                            'from_pc': 'first',
                            'min_clust_size': 10},
 'planes_segmentation': {'algo': 'svd_filter'}}
Random seed set as 123


Internal IDs100.0%|██████████| 61/61 [32:58<00:00, 32.43s/it]


Results were saved to "CARLA-downsampling_vs0.6.pkl".


# Downsampling & upsampling

## `voxel_size = 0.2`

In [6]:
cfg = load_config("downsampling_upsampling_vs0.2")
run_CARLA_seq5(SEED, cfg)

{'metrics': {'mean_map_metric': {'min_nn': 6},
             'mom': {'input': 'whole', 'min_nn': 6}},
 'normals_estimation': {'max_nn': 30, 'max_radius': 1.0},
 'orth_planes_extraction': {'affinity': 'cosine',
                            'dist_threshold': 0.1,
                            'downsample': {'upsample': True,
                                           'use': True,
                                           'voxel_size': 0.2},
                            'from_pc': 'first',
                            'min_clust_size': 10},
 'planes_segmentation': {'algo': 'svd_filter'}}
Random seed set as 123


Internal IDs: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61/61 [30:46<00:00, 30.28s/it]


Results were saved to "CARLA-downsampling_upsampling_vs0.2.pkl".


С оптимизированной метрикой МоМ посчиталось за 30 минут 46 секунд!

## `voxel_size = 0.3`

In [4]:
seed_everything(1)

with open('configs/mom_pipeline/downsampling_upsampling_vs0.3.yaml', 'r') as file:
    config = EasyDict(yaml.safe_load(file))

config

Random seed set as 1


{'normals_estimation': {'max_radius': 1.0, 'max_nn': 30},
 'orth_planes_extraction': {'min_clust_size': 10,
  'affinity': 'cosine',
  'dist_threshold': 0.1,
  'from_pc': 'first',
  'downsample': {'use': True, 'voxel_size': 0.3, 'upsample': True}},
 'planes_segmentation': {'algo': 'svd_filter'},
 'metrics': {'mom': {'min_nn': 6, 'input': 'whole'},
  'mean_map_metric': {'min_nn': 6}}}

In [ ]:
seed_everything(123)

prepare_func = prepare_data

map_size = 10
N_samples = 5
cov_scaler = 10

results = {}
for internal_id in tqdm(range(0, 2422, 40), "Internal IDs"):
    start_id, end_id = internal_id, internal_id + map_size

    data = CARLA_dataset(start_id=start_id, end_id=end_id, dataset_coords_type="world")

    pcs = data.clds
    T_gt = data.poses

    pcs, T_gt, orth_planes, orth_planes_cld_ind, details = prepare_func(
        pcs, T_gt, config)
    details["orth_planes"] = orth_planes
    details["orth_planes_cld_ind"] = orth_planes_cld_ind

    tips = EasyDict(orth_list=orth_planes, config=config)

    save_dict = sampling_pipeline(
        pcs=pcs, T_gt=T_gt, map_tips=tips,
        N_samples=N_samples, cov_scaler=cov_scaler,
        orth_planes_cld_ind=orth_planes_cld_ind
    )

    results[internal_id, map_size] = {**details, **save_dict}

voxel_size = config.orth_planes_extraction.downsample.voxel_size
with open(f"CARLA-downsampling_upsampling_vs{voxel_size}.pkl", 'wb') as sfile:
    pickle.dump(results, sfile)

С оптимизированной метрикой МоМ посчиталось за 30 минут 54 секунд!

## `voxel_size = 0.4`

In [8]:
seed_everything(1)

with open('configs/mom_pipeline/downsampling_upsampling_vs0.4.yaml', 'r') as file:
    config = EasyDict(yaml.safe_load(file))

config

Random seed set as 1


{'normals_estimation': {'max_radius': 1.0, 'max_nn': 30},
 'orth_planes_extraction': {'min_clust_size': 10,
  'affinity': 'cosine',
  'dist_threshold': 0.1,
  'from_pc': 'first',
  'downsample': {'use': True, 'voxel_size': 0.4, 'upsample': True}},
 'planes_segmentation': {'algo': 'svd_filter'},
 'metrics': {'mom': {'min_nn': 6, 'input': 'whole'},
  'mean_map_metric': {'min_nn': 6}}}

In [ ]:
seed_everything(123)

prepare_func = prepare_data

map_size = 10
N_samples = 5
cov_scaler = 10

results = {}
for internal_id in tqdm(range(0, 2422, 40), "Internal IDs"):
    start_id, end_id = internal_id, internal_id + map_size

    data = CARLA_dataset(start_id=start_id, end_id=end_id, dataset_coords_type="world")

    pcs = data.clds
    T_gt = data.poses

    pcs, T_gt, orth_planes, orth_planes_cld_ind, details = prepare_func(
        pcs, T_gt, config)
    details["orth_planes"] = orth_planes
    details["orth_planes_cld_ind"] = orth_planes_cld_ind

    tips = EasyDict(orth_list=orth_planes, config=config)

    save_dict = sampling_pipeline(
        pcs=pcs, T_gt=T_gt, map_tips=tips,
        N_samples=N_samples, cov_scaler=cov_scaler,
        orth_planes_cld_ind=orth_planes_cld_ind
    )

    results[internal_id, map_size] = {**details, **save_dict}

voxel_size = config.orth_planes_extraction.downsample.voxel_size
with open(f"CARLA-downsampling_upsampling_vs{voxel_size}.pkl", 'wb') as sfile:
    pickle.dump(results, sfile)

Считалось 2 часа 3 минуты. Тут метрика МоМ была не оптимизирована!

С оптимизированной метрикой МоМ посчиталось за 30 минут 11 секунд!

## `voxel_size = 0.6`

In [7]:
seed_everything(1)

with open('configs/mom_pipeline/downsampling_upsampling_vs0.6.yaml', 'r') as file:
    config = EasyDict(yaml.safe_load(file))

config

Random seed set as 1


{'normals_estimation': {'max_radius': 1.0, 'max_nn': 30},
 'orth_planes_extraction': {'min_clust_size': 10,
  'affinity': 'cosine',
  'dist_threshold': 0.1,
  'from_pc': 'first',
  'downsample': {'use': True, 'voxel_size': 0.6, 'upsample': True}},
 'planes_segmentation': {'algo': 'svd_filter'},
 'metrics': {'mom': {'min_nn': 6, 'input': 'whole'},
  'mean_map_metric': {'min_nn': 6}}}

In [ ]:
seed_everything(123)

prepare_func = prepare_data

map_size = 10
N_samples = 5
cov_scaler = 10

results = {}
for internal_id in tqdm(range(0, 2422, 40), "Internal IDs"):
    start_id, end_id = internal_id, internal_id + map_size

    data = CARLA_dataset(start_id=start_id, end_id=end_id, dataset_coords_type="world")

    pcs = data.clds
    T_gt = data.poses

    pcs, T_gt, orth_planes, orth_planes_cld_ind, details = prepare_func(
        pcs, T_gt, config)
    details["orth_planes"] = orth_planes
    details["orth_planes_cld_ind"] = orth_planes_cld_ind

    tips = EasyDict(orth_list=orth_planes, config=config)

    save_dict = sampling_pipeline(
        pcs=pcs, T_gt=T_gt, map_tips=tips,
        N_samples=N_samples, cov_scaler=cov_scaler,
        orth_planes_cld_ind=orth_planes_cld_ind
    )

    results[internal_id, map_size] = {**details, **save_dict}

voxel_size = config.orth_planes_extraction.downsample.voxel_size
with open(f"CARLA-downsampling_upsampling_vs{voxel_size}.pkl", 'wb') as sfile:
    pickle.dump(results, sfile)

С оптимизированной метрикой МоМ посчиталось за 22 минуты 27 секунд.

In [5]:
with open(f"CARLA-downsampling_upsampling_vs{0.6}.pkl", 'rb') as sfile:
    results_0p6 = pickle.load(sfile)

## `voxel_size = 0.8`

In [10]:
seed_everything(1)

with open('configs/mom_pipeline/downsampling_upsampling_vs0.8.yaml', 'r') as file:
    config = EasyDict(yaml.safe_load(file))

config

Random seed set as 1


{'normals_estimation': {'max_radius': 1.0, 'max_nn': 30},
 'orth_planes_extraction': {'min_clust_size': 10,
  'affinity': 'cosine',
  'dist_threshold': 0.1,
  'from_pc': 'first',
  'downsample': {'use': True, 'voxel_size': 0.8, 'upsample': True}},
 'planes_segmentation': {'algo': 'svd_filter'},
 'metrics': {'mom': {'min_nn': 6, 'input': 'whole'},
  'mean_map_metric': {'min_nn': 6}}}

In [ ]:
seed_everything(123)

prepare_func = prepare_data

map_size = 10
N_samples = 5
cov_scaler = 10

results = {}
for internal_id in tqdm(range(0, 2422, 40), "Internal IDs"):
    start_id, end_id = internal_id, internal_id + map_size

    data = CARLA_dataset(start_id=start_id, end_id=end_id, dataset_coords_type="world")

    pcs = data.clds
    T_gt = data.poses

    pcs, T_gt, orth_planes, orth_planes_cld_ind, details = prepare_func(
        pcs, T_gt, config)
    details["orth_planes"] = orth_planes
    details["orth_planes_cld_ind"] = orth_planes_cld_ind

    tips = EasyDict(orth_list=orth_planes, config=config)

    save_dict = sampling_pipeline(
        pcs=pcs, T_gt=T_gt, map_tips=tips,
        N_samples=N_samples, cov_scaler=cov_scaler,
        orth_planes_cld_ind=orth_planes_cld_ind
    )

    results[internal_id, map_size] = {**details, **save_dict}

voxel_size = config.orth_planes_extraction.downsample.voxel_size
with open(f"CARLA-downsampling_upsampling_vs{voxel_size}.pkl", 'wb') as sfile:
    pickle.dump(results, sfile)

С оптимизированной метрикой МоМ посчиталось за 11 минут 49 секунд.

## `voxel_size = 1.0`

In [8]:
cfg = load_config("downsampling_upsampling_vs1.0")
run_CARLA_seq5(SEED, cfg)

{'metrics': {'mean_map_metric': {'min_nn': 6},
             'mom': {'input': 'whole', 'min_nn': 6}},
 'normals_estimation': {'max_nn': 30, 'max_radius': 1.0},
 'orth_planes_extraction': {'affinity': 'cosine',
                            'dist_threshold': 0.1,
                            'downsample': {'upsample': True,
                                           'use': True,
                                           'voxel_size': 1.0},
                            'from_pc': 'first',
                            'min_clust_size': 10},
 'planes_segmentation': {'algo': 'svd_filter'}}
Random seed set as 123


Internal IDs100.0%|██████████| 61/61 [09:05<00:00,  8.95s/it]

Results were saved to "CARLA-downsampling_upsampling_vs1.0.pkl".


С оптимизированной метрикой МоМ посчиталось за 9 минут 5 секунд!

## `voxel_size = 1.5`

In [7]:
cfg = load_config("downsampling_upsampling_vs1.5")
run_CARLA_seq5(SEED, cfg)

{'metrics': {'mean_map_metric': {'min_nn': 6},
             'mom': {'input': 'whole', 'min_nn': 6}},
 'normals_estimation': {'max_nn': 30, 'max_radius': 1.0},
 'orth_planes_extraction': {'affinity': 'cosine',
                            'dist_threshold': 0.1,
                            'downsample': {'upsample': True,
                                           'use': True,
                                           'voxel_size': 1.5},
                            'from_pc': 'first',
                            'min_clust_size': 10},
 'planes_segmentation': {'algo': 'svd_filter'}}
Random seed set as 123


Internal IDs100.0%|██████████| 61/61 [09:05<00:00,  8.94s/it]


Results were saved to "CARLA-downsampling_upsampling_vs1.5.pkl".


С оптимизированной метрикой МоМ посчиталось за 9 минут 5 секунд!

# Downsampling

## `voxel_size = 0.2`

In [6]:
cfg = load_config("downsampling_vs0.2")
run_CARLA_seq5(SEED, cfg)

{'metrics': {'mean_map_metric': {'min_nn': 6},
             'mom': {'input': 'whole', 'min_nn': 6}},
 'normals_estimation': {'max_nn': 30, 'max_radius': 1.0},
 'orth_planes_extraction': {'affinity': 'cosine',
                            'dist_threshold': 0.1,
                            'downsample': {'upsample': False,
                                           'use': True,
                                           'voxel_size': 0.2},
                            'from_pc': 'first',
                            'min_clust_size': 10},
 'planes_segmentation': {'algo': 'svd_filter'}}
Random seed set as 123


Internal IDs100.0%|██████████| 61/61 [17:48<00:00, 17.51s/it]


Results were saved to "CARLA-downsampling_vs0.2.pkl".


С оптимизированной метрикой МоМ посчиталось за 17 минут 48 секунд!

## `voxel_size = 0.3`

In [6]:
seed_everything(1)

with open('configs/mom_pipeline/downsampling_vs0.3.yaml', 'r') as file:
    config = EasyDict(yaml.safe_load(file))

config

Random seed set as 1


{'normals_estimation': {'max_radius': 1.0, 'max_nn': 30},
 'orth_planes_extraction': {'min_clust_size': 10,
  'affinity': 'cosine',
  'dist_threshold': 0.1,
  'from_pc': 'first',
  'downsample': {'use': True, 'voxel_size': 0.3, 'upsample': False}},
 'planes_segmentation': {'algo': 'svd_filter'},
 'metrics': {'mom': {'min_nn': 6, 'input': 'whole'},
  'mean_map_metric': {'min_nn': 6}}}

In [ ]:
seed_everything(123)

prepare_func = prepare_data

map_size = 10
N_samples = 5
cov_scaler = 10

results = {}
for internal_id in tqdm(range(0, 2422, 40), "Internal IDs"):
    start_id, end_id = internal_id, internal_id + map_size

    data = CARLA_dataset(start_id=start_id, end_id=end_id, dataset_coords_type="world")

    pcs = data.clds
    T_gt = data.poses

    pcs, T_gt, orth_planes, orth_planes_cld_ind, details = prepare_func(
        pcs, T_gt, config)
    details["orth_planes"] = orth_planes
    details["orth_planes_cld_ind"] = orth_planes_cld_ind

    tips = EasyDict(orth_list=orth_planes, config=config)

    save_dict = sampling_pipeline(
        pcs=pcs, T_gt=T_gt, map_tips=tips,
        N_samples=N_samples, cov_scaler=cov_scaler,
        orth_planes_cld_ind=orth_planes_cld_ind
    )

    results[internal_id, map_size] = {**details, **save_dict}

voxel_size = config.orth_planes_extraction.downsample.voxel_size
with open(f"CARLA-downsampling_vs{voxel_size}.pkl", 'wb') as sfile:
    pickle.dump(results, sfile)

С оптимизированной метрикой МоМ посчиталось за 12 минут 49 секунд!

## `voxel_size = 0.4`

In [13]:
seed_everything(1)

with open('configs/mom_pipeline/downsampling_vs0.4.yaml', 'r') as file:
    config = EasyDict(yaml.safe_load(file))

config

Random seed set as 1


{'normals_estimation': {'max_radius': 1.0, 'max_nn': 30},
 'orth_planes_extraction': {'min_clust_size': 10,
  'affinity': 'cosine',
  'dist_threshold': 0.1,
  'from_pc': 'first',
  'downsample': {'use': True, 'voxel_size': 0.4, 'upsample': False}},
 'planes_segmentation': {'algo': 'svd_filter'},
 'metrics': {'mom': {'min_nn': 6, 'input': 'whole'},
  'mean_map_metric': {'min_nn': 6}}}

In [ ]:
seed_everything(123)

prepare_func = prepare_data

map_size = 10
N_samples = 5
cov_scaler = 10

results = {}
for internal_id in tqdm(range(0, 2422, 40), "Internal IDs"):
    start_id, end_id = internal_id, internal_id + map_size

    data = CARLA_dataset(start_id=start_id, end_id=end_id, dataset_coords_type="world")

    pcs = data.clds
    T_gt = data.poses

    pcs, T_gt, orth_planes, orth_planes_cld_ind, details = prepare_func(
        pcs, T_gt, config)
    details["orth_planes"] = orth_planes
    details["orth_planes_cld_ind"] = orth_planes_cld_ind

    tips = EasyDict(orth_list=orth_planes, config=config)

    save_dict = sampling_pipeline(
        pcs=pcs, T_gt=T_gt, map_tips=tips,
        N_samples=N_samples, cov_scaler=cov_scaler,
        orth_planes_cld_ind=orth_planes_cld_ind
    )

    results[internal_id, map_size] = {**details, **save_dict}

voxel_size = config.orth_planes_extraction.downsample.voxel_size
with open(f"CARLA-downsampling_vs{voxel_size}.pkl", 'wb') as sfile:
    pickle.dump(results, sfile)

С оптимизированной метрикой МоМ посчиталось за 10 минут 0 секунд!

## `voxel_size = 0.6`

In [15]:
seed_everything(1)

with open('configs/mom_pipeline/downsampling_vs0.6.yaml', 'r') as file:
    config = EasyDict(yaml.safe_load(file))

config

Random seed set as 1


{'normals_estimation': {'max_radius': 1.0, 'max_nn': 30},
 'orth_planes_extraction': {'min_clust_size': 10,
  'affinity': 'cosine',
  'dist_threshold': 0.1,
  'from_pc': 'first',
  'downsample': {'use': True, 'voxel_size': 0.6, 'upsample': False}},
 'planes_segmentation': {'algo': 'svd_filter'},
 'metrics': {'mom': {'min_nn': 6, 'input': 'whole'},
  'mean_map_metric': {'min_nn': 6}}}

In [ ]:
seed_everything(123)

prepare_func = prepare_data

map_size = 10
N_samples = 5
cov_scaler = 10

results = {}
for internal_id in tqdm(range(0, 2422, 40), "Internal IDs"):
    start_id, end_id = internal_id, internal_id + map_size

    data = CARLA_dataset(start_id=start_id, end_id=end_id, dataset_coords_type="world")

    pcs = data.clds
    T_gt = data.poses

    pcs, T_gt, orth_planes, orth_planes_cld_ind, details = prepare_func(
        pcs, T_gt, config)
    details["orth_planes"] = orth_planes
    details["orth_planes_cld_ind"] = orth_planes_cld_ind

    tips = EasyDict(orth_list=orth_planes, config=config)

    save_dict = sampling_pipeline(
        pcs=pcs, T_gt=T_gt, map_tips=tips,
        N_samples=N_samples, cov_scaler=cov_scaler,
        orth_planes_cld_ind=orth_planes_cld_ind
    )

    results[internal_id, map_size] = {**details, **save_dict}

voxel_size = config.orth_planes_extraction.downsample.voxel_size
with open(f"CARLA-downsampling_vs{voxel_size}.pkl", 'wb') as sfile:
    pickle.dump(results, sfile)

С оптимизированной метрикой МоМ посчиталось за 7 минут 19 секунд!

## `voxel_size = 0.8`

In [7]:
cfg = load_config("downsampling_vs0.8")
run_CARLA_seq5(SEED, cfg)

{'metrics': {'mean_map_metric': {'min_nn': 6},
             'mom': {'input': 'whole', 'min_nn': 6}},
 'normals_estimation': {'max_nn': 30, 'max_radius': 1.0},
 'orth_planes_extraction': {'affinity': 'cosine',
                            'dist_threshold': 0.1,
                            'downsample': {'upsample': False,
                                           'use': True,
                                           'voxel_size': 0.8},
                            'from_pc': 'first',
                            'min_clust_size': 10},
 'planes_segmentation': {'algo': 'svd_filter'}}
Random seed set as 123


Internal IDs100.0%|██████████| 61/61 [06:24<00:00,  6.31s/it]


Results were saved to "CARLA-downsampling_vs0.8.pkl".


С оптимизированной метрикой МоМ посчиталось за 6 минут 24 секунд!

In [17]:
seed_everything(1)

with open('configs/mom_pipeline/downsampling_vs0.8.yaml', 'r') as file:
    config = EasyDict(yaml.safe_load(file))

config

Random seed set as 1


{'normals_estimation': {'max_radius': 1.0, 'max_nn': 30},
 'orth_planes_extraction': {'min_clust_size': 10,
  'affinity': 'cosine',
  'dist_threshold': 0.1,
  'from_pc': 'first',
  'downsample': {'use': True, 'voxel_size': 0.8, 'upsample': False}},
 'planes_segmentation': {'algo': 'svd_filter'},
 'metrics': {'mom': {'min_nn': 6, 'input': 'whole'},
  'mean_map_metric': {'min_nn': 6}}}

In [ ]:
seed_everything(123)

prepare_func = prepare_data

map_size = 10
N_samples = 5
cov_scaler = 10

results = {}
for internal_id in tqdm(range(0, 2422, 40), "Internal IDs"):
    start_id, end_id = internal_id, internal_id + map_size

    data = CARLA_dataset(start_id=start_id, end_id=end_id, dataset_coords_type="world")

    pcs = data.clds
    T_gt = data.poses

    pcs, T_gt, orth_planes, orth_planes_cld_ind, details = prepare_func(
        pcs, T_gt, config)
    details["orth_planes"] = orth_planes
    details["orth_planes_cld_ind"] = orth_planes_cld_ind

    tips = EasyDict(orth_list=orth_planes, config=config)

    save_dict = sampling_pipeline(
        pcs=pcs, T_gt=T_gt, map_tips=tips,
        N_samples=N_samples, cov_scaler=cov_scaler,
        orth_planes_cld_ind=orth_planes_cld_ind
    )

    results[internal_id, map_size] = {**details, **save_dict}

voxel_size = config.orth_planes_extraction.downsample.voxel_size
with open(f"CARLA-downsampling_vs{voxel_size}.pkl", 'wb') as sfile:
    pickle.dump(results, sfile)

С оптимизированной метрикой МоМ посчиталось за 6 минут 31 секунд!

## `voxel_size = 1.0`

In [9]:
cfg = load_config("downsampling_vs1.0")
run_CARLA_seq5(SEED, cfg)

{'metrics': {'mean_map_metric': {'min_nn': 6},
             'mom': {'input': 'whole', 'min_nn': 6}},
 'normals_estimation': {'max_nn': 30, 'max_radius': 1.0},
 'orth_planes_extraction': {'affinity': 'cosine',
                            'dist_threshold': 0.1,
                            'downsample': {'upsample': False,
                                           'use': True,
                                           'voxel_size': 1.0},
                            'from_pc': 'first',
                            'min_clust_size': 10},
 'planes_segmentation': {'algo': 'svd_filter'}}
Random seed set as 123


Internal IDs100.0%|██████████| 61/61 [06:05<00:00,  5.99s/it]

Results were saved to "CARLA-downsampling_vs1.0.pkl".


С оптимизированной метрикой МоМ посчиталось за 6 минут 5 секунд!

## `voxel_size = 1.5`

In [8]:
cfg = load_config("downsampling_vs1.5")
run_CARLA_seq5(SEED, cfg)

{'metrics': {'mean_map_metric': {'min_nn': 6},
             'mom': {'input': 'whole', 'min_nn': 6}},
 'normals_estimation': {'max_nn': 30, 'max_radius': 1.0},
 'orth_planes_extraction': {'affinity': 'cosine',
                            'dist_threshold': 0.1,
                            'downsample': {'upsample': False,
                                           'use': True,
                                           'voxel_size': 1.5},
                            'from_pc': 'first',
                            'min_clust_size': 10},
 'planes_segmentation': {'algo': 'svd_filter'}}
Random seed set as 123


Internal IDs100.0%|██████████| 61/61 [05:57<00:00,  5.86s/it]

Results were saved to "CARLA-downsampling_vs1.5.pkl".


С оптимизированной метрикой МоМ посчиталось за 5 минут 57 секунд!